In [1]:
import torch
from torch import nn
import torchvision

import matplotlib.pyplot as plt
from torchvision import transforms
from torchinfo import summary

from pathlib import Path

from going_modular.going_modular import engine, data_setup

d:\Work\Coding\Machine-Learning\PyTorch\going_modular\going_modular\engine.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Device agnostic code

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
## Path

dataset_path = Path("datasets/pizza_steak_sushi")
train_dir = dataset_path / "train"
test_dir = dataset_path / "test"

## Normalize

normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std = [0.229, 0.224, 0.225]
)

## Transform

# custom transform

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225]
    )
])

# auto transform

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transform = weights.transforms()



train_dataloader, test_dataloader, classes = data_setup.create_dataloaders(train_dir=train_dir,
                                                                           test_dir=test_dir,
                                                                           transform=data_transform,
                                                                           batch_size=32,
                                                                           num_workers=2
                                                                           )

train_dataloader, test_dataloader, classes

(<torch.utils.data.dataloader.DataLoader at 0x22a120e0550>,
 ['pizza', 'steak', 'sushi'])

## Use both tranform to understand the difference between them

In [8]:
auto_transform, data_transform

(ImageClassification(
     crop_size=[224]
     resize_size=[256]
     mean=[0.485, 0.456, 0.406]
     std=[0.229, 0.224, 0.225]
     interpolation=InterpolationMode.BICUBIC
 ),
 Compose(
     Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
     ToTensor()
     Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
 ))

## Let's load model

In [9]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [13]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [19]:
summary(model,
        input_size=[32, 3, 245, 245],
        col_names=['input_size', "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 245, 245]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 245, 245]    [32, 1280, 8, 8]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 245, 245]    [32, 32, 123, 123]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 245, 245]    [32, 32, 123, 123]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 123, 123]   [32, 32, 123, 123]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 123, 123]   [32, 32, 123, 123]   --                   --
│    └─Sequential (1)                                        [32, 32, 123, 123]   [32, 16, 123

## Freeze the pre trained model features section

In [20]:
for param in model.features.parameters():

    param.requires_grad = False